# 最終課題

In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time

設定

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# 武蔵野大学のトップページURL
url = 'https://www.musashino-u.ac.jp/'
base_domain = 'www.musashino-u.ac.jp'

print(f"アクセス先URL: {url}")
print(f"対象ドメイン: {base_domain}")


アクセス先URL: https://www.musashino-u.ac.jp/
対象ドメイン: www.musashino-u.ac.jp


補助関数の定義

In [5]:
def is_same_domain(url, domain):
    parsed = urlparse(url)
    return parsed.netloc == domain or parsed.netloc == domain.replace('www.', '')

def should_skip(url):

    skip_extensions = ('.pdf', '.png', '.jpg', '.jpeg', '.gif', '.zip', 
                       '.mp4', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx')
    return url.lower().endswith(skip_extensions)

# テスト
test_url = "https://www.musashino-u.ac.jp/academics/"
print(f" {test_url} は同一ドメイン → {is_same_domain(test_url, base_domain)}")


 https://www.musashino-u.ac.jp/academics/ は同一ドメイン → True


全500ページのスクレイピング

In [6]:
# 変数の初期化
visited = set()
to_visit = [url]
sitemap = {}
max_pages = 500

# メインループ
while to_visit and len(visited) < max_pages:
    current_url = to_visit.pop(0)
    
    # スキップ条件
    if current_url in visited or should_skip(current_url) or '?' in current_url:
        continue
    
    try:
        # 進捗表示
        if len(visited) % 10 == 0 and len(visited) > 0:
            print(f"処理済み: {len(visited)}ページ")
        
        # HTTPリクエスト
        res = requests.get(current_url, headers=headers, timeout=10)
        res.encoding = res.apparent_encoding
        
        if res.status_code != 200:
            continue
        
        visited.add(current_url)
        
        # HTML解析
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # タイトル取得
        title_tag = soup.find('title')
        title_text = title_tag.string.strip() if title_tag and title_tag.string else "No Title"
        
        # サイトマップに追加
        sitemap[current_url] = title_text
        
        # リンク収集
        for a in soup.find_all('a', href=True):
            full_url = urljoin(current_url, a['href']).split('#')[0]
            
            if '?' not in full_url and is_same_domain(full_url, base_domain):
                if full_url not in visited and full_url not in to_visit:
                    to_visit.append(full_url)
        
        # サーバー負荷軽減
        time.sleep(1)
        
    except:
        continue

print(f"訪問ページ数: {len(sitemap)}")


処理済み: 10ページ
処理済み: 20ページ
処理済み: 30ページ
処理済み: 40ページ
処理済み: 50ページ
処理済み: 60ページ
処理済み: 70ページ
処理済み: 80ページ
処理済み: 90ページ
処理済み: 100ページ
処理済み: 110ページ
処理済み: 120ページ
処理済み: 130ページ
処理済み: 140ページ
処理済み: 150ページ
処理済み: 160ページ
処理済み: 170ページ
処理済み: 180ページ
処理済み: 190ページ
処理済み: 200ページ
処理済み: 210ページ
処理済み: 220ページ
処理済み: 230ページ
処理済み: 240ページ
処理済み: 250ページ
処理済み: 260ページ
処理済み: 270ページ
処理済み: 280ページ
処理済み: 290ページ
処理済み: 300ページ
処理済み: 310ページ
処理済み: 320ページ
処理済み: 330ページ
処理済み: 340ページ
処理済み: 350ページ
処理済み: 360ページ
処理済み: 370ページ
処理済み: 380ページ
処理済み: 390ページ
処理済み: 400ページ
処理済み: 410ページ
処理済み: 420ページ
処理済み: 430ページ
処理済み: 440ページ
処理済み: 450ページ
処理済み: 460ページ
処理済み: 470ページ
処理済み: 480ページ
処理済み: 490ページ
訪問ページ数: 500


# サイトマップ表示

In [7]:
print(sitemap)

{'https://www.musashino-u.ac.jp/': '武蔵野大学', 'https://www.musashino-u.ac.jp/access.html': '交通アクセス | 武蔵野大学', 'https://www.musashino-u.ac.jp/admission/request.html': '資料請求 | 入試情報 | 武蔵野大学', 'https://www.musashino-u.ac.jp/contact.html': 'お問い合わせ | 武蔵野大学', 'https://www.musashino-u.ac.jp/prospective-students.html': '武蔵野大学で学びたい方 | 武蔵野大学', 'https://www.musashino-u.ac.jp/students.html': '在学生の方 | 武蔵野大学', 'https://www.musashino-u.ac.jp/alumni.html': '卒業生の方 | 武蔵野大学', 'https://www.musashino-u.ac.jp/parents.html': '保護者の方 | 武蔵野大学', 'https://www.musashino-u.ac.jp/business.html': '企業・研究者の方 | 武蔵野大学', 'https://www.musashino-u.ac.jp/guide/': '大学案内 | 武蔵野大学', 'https://www.musashino-u.ac.jp/guide/profile/': '大学紹介 | 大学案内 | 武蔵野大学', 'https://www.musashino-u.ac.jp/guide/activities/': '大学の取り組み | 大学案内 | 武蔵野大学', 'https://www.musashino-u.ac.jp/guide/campus/': 'キャンパス | 大学案内 | 武蔵野大学', 'https://www.musashino-u.ac.jp/guide/facility/': '附置機関・センター・附属施設 | 大学案内 | 武蔵野大学', 'https://www.musashino-u.ac.jp/guide/information/': '情報

In [9]:
print("サイトマップ一覧")

for i, (url, title) in enumerate(sitemap.items(), 1):
    print(f"{i}. {title}")
    print(f"   URL: {url}\n")

print(f"合計: {len(sitemap)}ページ")

サイトマップ一覧
1. 武蔵野大学
   URL: https://www.musashino-u.ac.jp/

2. 交通アクセス | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/access.html

3. 資料請求 | 入試情報 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/admission/request.html

4. お問い合わせ | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/contact.html

5. 武蔵野大学で学びたい方 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/prospective-students.html

6. 在学生の方 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/students.html

7. 卒業生の方 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/alumni.html

8. 保護者の方 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/parents.html

9. 企業・研究者の方 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/business.html

10. 大学案内 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/guide/

11. 大学紹介 | 大学案内 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/guide/profile/

12. 大学の取り組み | 大学案内 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/guide/activities/

13. キャンパス | 大学案内 | 武蔵野大学
   URL: https://www.musashino-u.ac.jp/guide/campus/

14. 附置機関・センター・附属施設 | 大学案内 | 武蔵野大学
   URL: https